In [1]:
import coiled
import dask
import dask.dataframe as dd

In [2]:
cluster = coiled.Cluster(name="powers-benchmark2", n_workers=3)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/rich/live.py:22
1: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Using existing cluster: 'powers-benchmark2'

In [3]:
client = dask.distributed.Client(cluster)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| dask        | 2021.11.2 | 2022.01.0 | 2022.01.0 |
| distributed | 2021.11.2 | 2022.01.0 | 2022.01.0 |
| numpy       | 1.22.0    | 1.21.5    | 1.21.5    |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


## Coiled Quickstart

In [9]:
%%time

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
        "store_and_fwd_flag": "category",
        "PULocationID": "UInt16",
        "DOLocationID": "UInt16",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
)

df.groupby("passenger_count").tip_amount.mean().compute()

CPU times: user 1.06 s, sys: 197 ms, total: 1.26 s
Wall time: 2min


passenger_count
0    2.122789
1    2.206790
2    2.214306
3    2.137775
4    2.023804
5    2.235441
6    2.221105
7    6.675962
8    7.111625
9    7.377822
Name: tip_amount, dtype: float64

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/comm/tcp.py", line 398, in connect
    stream = await self.client.connect(
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/asyncio/task

## Same query with coiled-datasets

In [7]:
%%time

ddf = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2019",
    engine="pyarrow",
    columns=["passenger_count", "tip_amount"],
)
ddf.groupby("passenger_count").tip_amount.mean().compute()

CPU times: user 343 ms, sys: 121 ms, total: 464 ms
Wall time: 16 s


passenger_count
0.0    2.122789
1.0    2.206790
2.0    2.214306
3.0    2.137775
4.0    2.023804
5.0    2.235441
6.0    2.221105
7.0    6.675962
8.0    7.111625
9.0    7.377822
Name: tip_amount, dtype: float64

## Same query with coiled-datasets w/ metadata

In [8]:
%%time

ddf = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc-with-metadata/2019",
    engine="pyarrow",
    columns=["passenger_count", "tip_amount"],
)
ddf.groupby("passenger_count").tip_amount.mean().compute()

CPU times: user 220 ms, sys: 25.2 ms, total: 245 ms
Wall time: 13.6 s


passenger_count
0.0    2.122789
1.0    2.206790
2.0    2.214306
3.0    2.137775
4.0    2.023804
5.0    2.235441
6.0    2.221105
7.0    6.675962
8.0    7.111625
9.0    7.377822
Name: tip_amount, dtype: float64

In [15]:
client.restart()

<Client: 'tls://10.4.8.186:8786' processes=3 threads=6, memory=22.43 GiB>